In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.misc import imread
import numpy as np
import os
import tensorflow as tf
import glob
import random
import shutil

IMG_width = 320
IMG_height = 240
IMG_channel = 1
DATASET_PATH = "./dataset_jpg_gray"
kinds = ["train", "validation", "test"]
train_img = np.array([], dtype="float32")
train_label = np.array([], dtype="int8")
validation_img = np.array([], dtype="float32")
validation_label = np.array([], dtype="int8")
test_img = np.array([], dtype="float32")
test_label = np.array([], dtype="int8")
(label_train, filename_train) = (0, 0)
(label_validation, filename_validation) = (0, 0)
(label_test, filename_test) = (0, 0)


def file_info(category_name, dataset_path=DATASET_PATH):
    # 디렉토리 상의 파일경로와 파일의 제일 앞에 매겨진 숫자정보(라벨 정보)를 긁어서 반환
    full_path =  dataset_path + '/' + category_name + '/' + '*.jpg'
    image_filenames = glob.glob(full_path)
    filename = []
    label = []
    for image_filename in image_filenames:
        filename.append(image_filename.split("/")[3])
        label.append(image_filename.split("/")[3].split("-")[0])
    return (label, filename)


def display_image(image, label):
    %matplotlib
    inline

    fig = plt.figure(figsize=(20, 5))
    for i in range(5):
        print(label[i])
        img = image[i]
        img = img.reshape(240, 320, 1)
        img.shape = (240, 320, 1)
        plt.subplot(150 + (i + 1))
        plt.imshow(img)


def data_processing_about_train(idx, kind):
    for i in range(idx):
        global train_img
        global train_label

        (labels, filenames) = file_info(kind)
        random_value = random.randrange(0, len(filenames))
        filename = filenames[random_value]
        label = labels[random_value]
        #         label = _change_one_hot_label(train_label, label)
        #         label = labels[random_value]
        #         image_display(kinds, filename)
        full_path = ( DATASET_PATH + '/' + kind + '/' + filename)
        image = imread(full_path)
        image = image.reshape(IMG_width * IMG_height * IMG_channel) / 255.0
        train_img = np.append(train_img, image)
        label = _change_one_hot_label(label)
        train_label = np.append(train_label, label)
        # file move
        # 학습한 데이터는 이동시킴
        print("---------")
        print(len(filenames))
        print(filename)
        src =  DATASET_PATH + '/' + kind + '/'
        dir =  DATASET_PATH + '/' + 'tmp' + '/' + kind + '/'
        shutil.move(src + filename, dir + filename)


def data_processing_about_validation(idx, kind):
    for i in range(idx):
        global validation_img
        global validation_label

        (labels, filenames) = file_info(kind)
        random_value = random.randrange(0, len(filenames))
        filename = filenames[random_value]
        label = labels[random_value]
        #         label = _change_one_hot_label(train_label, label)
        #         label = labels[random_value]
        #         image_display(kinds, filename)
        full_path = ( DATASET_PATH + '/' + kind + '/' + filename)
        image = imread(full_path)
        image = image.reshape(IMG_width * IMG_height * IMG_channel) / 255.0
        validation_img = np.append(validation_img, image)
        label = _change_one_hot_label(label)
        validation_label = np.append(validation_label, label)
        # file move
        print("---------")
        print(len(filenames))
        print(filename)
        src =  DATASET_PATH + '/' + kind + '/'
        dir =  DATASET_PATH + '/' + 'tmp' + '/' + kind + '/'
        shutil.move(src + filename, dir + filename)


def data_processing_about_test(idx, kind):
    for i in range(idx):
        global test_img
        global test_label

        (labels, filenames) = file_info(kind)
        random_value = random.randrange(0, len(filenames))
        filename = filenames[random_value]
        label = labels[random_value]
        #         label = _change_one_hot_label(train_label, label)
        #         label = labels[random_value]
        #         image_display(kinds, filename)
        full_path = ( DATASET_PATH + '/' + kind + '/' + filename)
        image = imread(full_path)
        image = image.reshape(IMG_width * IMG_height * IMG_channel) / 255.0
        test_img = np.append(test_img, image)
        label = _change_one_hot_label(label)
        test_label = np.append(test_label, label)
        # file move
        print("---------")
        print(len(filenames))
        print(filename)
        src =  DATASET_PATH + '/' + kind + '/'
        dir =  DATASET_PATH + '/' + 'tmp' + '/' + kind + '/'
        shutil.move(src + filename, dir + filename)


def _change_one_hot_label(target_label):
    target_label = int(target_label)
    T = np.zeros((1, 3))
    T[0][target_label] = 1

    return T


def store_dataset(dataset_path=DATASET_PATH):
    global train_img
    global train_label
    global validation_img
    global validation_label
    global test_img
    global test_label

    (_, idx_train) = file_info("train")
    (_, idx_validation) = file_info("validation")
    (_, idx_test) = file_info("test")
    if len(idx_train) != 0 or len(idx_validation) != 0 or len(idx_test) != 0:
        data_processing_about_train(len(idx_train), "train")
        data_processing_about_validation(len(idx_validation), "validation")
        data_processing_about_test(len(idx_test), "test")

        # file로 쓰기
        train_img.tofile(dataset_path + '/' + 'train_img_dataset.txt')
        train_label.tofile(dataset_path + '/' + 'train_label.txt')
        validation_img.tofile(dataset_path + '/' + 'validation_img_dataset.txt')
        validation_label.tofile(dataset_path + '/' + 'validation_label.txt')
        test_img.tofile(dataset_path + '/' + 'test_img_dataset.txt')
        test_label.tofile(dataset_path + '/' + 'test_label.txt')

    else:
        print("[!] Already Finished Generateing Dataset. Please check directory.")





def load_dataset(dataset_path):
    (img_train, label_train) = (0,0)
    (img_validation, label_validation) = (0,0)
    (img_test, label_test) = (0,0)
    filenames = glob.glob(dataset_path + "/" + "*.txt")
    filename = []
    for filename in filenames:
        print(filename, )
        if filename.split("/")[6][-4:] == ".txt": # CAUTION : filename.split("/")[directory level][-4:]
            img_train = np.fromfile(open(dataset_path + "/" + "train_img_dataset.txt", 'rb')).reshape(451,IMG_width * IMG_height * IMG_channel)
            label_train = np.fromfile(open(dataset_path + "/" + "train_label.txt", 'rb')).reshape(451, 3)
            img_validation = np.fromfile(open(dataset_path + "/" + "validation_img_dataset.txt", 'rb')).reshape(65, IMG_width * IMG_height * IMG_channel)
            label_validation = np.fromfile(open(dataset_path + "/" + "validation_label.txt", 'rb')).reshape(65,3)
            img_test = np.fromfile(open(dataset_path + "/" + "test_img_dataset.txt", 'rb')).reshape(130,IMG_width * IMG_height * IMG_channel)
            label_test = np.fromfile(open(dataset_path + "/" + "test_label.txt", 'rb')).reshape(130, 3)
    return ((img_train, label_train), (img_validation, label_validation), (img_test, label_test))







In [2]:
import numpy as np

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x)
    return np.exp(x) / np.sum(np.exp(x))

def cross_entropy_error(y, t):
    #print(y.shape, t.shape)
    if y.ndim == 1:
        y = y.reshape(1, y.size)
        t = t.reshape(1, t.size)

    if t.size == y.size:
        t = t.argmax(axis=1)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out

    def backward(self, din):
        din[self.mask] = 0
        dx = din
        return dx

class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, din):
        dx = din * self.out * (1.0 - self.out)
        return dx

class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b        
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(self.x, self.W) + self.b
        return out

    def backward(self, din):
        dx = np.dot(din, self.W.T)
        self.dW = np.dot(self.x.T, din)
        self.db = np.sum(din, axis=0)
        return dx

class SoftmaxWithCrossEntropyLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
    
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, din=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / float(batch_size)        
        return dx

In [3]:
import sys, os
from collections import OrderedDict
from scipy import stats
from pandas import DataFrame

class TwoLayerNet2:
    def __init__(self, input_size, hidden_layer1_size, hidden_layer2_size, output_size, weight_init_std = 0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_layer1_size)
        self.params['b1'] = np.zeros(hidden_layer1_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_layer1_size, hidden_layer2_size)
        self.params['b2'] = np.zeros(hidden_layer2_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_layer2_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine3'] = Affine(self.params['W3'], self.params['b3'])

        self.lastLayer = SoftmaxWithCrossEntropyLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    def backpropagation_gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        din = 1
        din = self.lastLayer.backward(din)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            din = layer.backward(din)

        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        grads['W3'], grads['b3'] = self.layers['Affine3'].dW, self.layers['Affine3'].db

        return grads
    
    def learning(self, learning_rate, x_batch, t_batch):
        grads = self.backpropagation_gradient(x_batch, t_batch)
        for key in ('W1', 'b1', 'W2', 'b2', 'W3', 'b3'):
            self.params[key] -= learning_rate * grads[key]

In [9]:
import math
import time

(img_train, label_train), (img_validation, label_validation), (img_test, label_test) = load_dataset("/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata")


network = TwoLayerNet2(input_size=76800, hidden_layer1_size=128, hidden_layer2_size=128, output_size=3)

num_epochs = 50
train_size = img_train.shape[0]
batch_size = 10
learning_rate = 0.01

train_error_list = []
validation_error_list = []

test_accuracy_list = []
epoch_list = []

num_batch = math.ceil(train_size / batch_size)

start = time.time()
for i in range(num_epochs):
#     batch_mask = np.random.choice(train_size, batch_size)
#     x_batch = img_train[batch_mask]
#     t_batch = label_train[batch_mask]
#     network.learning(learning_rate, x_batch, t_batch)
    for j in range(num_batch):
        x_batch = img_train[j * batch_size : j * batch_size + batch_size]
        t_batch = label_train[j * batch_size : j * batch_size + batch_size]
        network.learning(learning_rate, x_batch, t_batch)

    epoch_list.append(i)
    
    train_loss = network.loss(x_batch, t_batch)
    train_error_list.append(train_loss)
    
    validation_loss = network.loss(img_validation, label_validation)
    validation_error_list.append(validation_loss)    
    
    test_accuracy = network.accuracy(img_test, label_test)
    test_accuracy_list.append(test_accuracy)
    
    print("Epoch: {0:5d}, Train Error: {1:7.5f}, Validation Error: {2:7.5f} - Test Accuracy: {3:7.5f}".format(
        i,
        train_loss,
        validation_loss,
        test_accuracy
    ))
end = time.time()
print("consumed time : ", int(end-start), "Second.")

/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/test_img_dataset.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/test_label.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/train_img_dataset.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/train_label.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/validation_img_dataset.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/validation_label.txt
Epoch:     0, Train Error: 1.10374, Validation Error: 1.07906 - Test Accuracy: 0.49231
Epoch:     1, Train Error: 1.08171, Validation Error: 1.06636 - Test Accuracy: 0.49231
Epoch:     2, Train Error: 1.01092, Validation Error: 1.05535 - Test Accuracy: 0.50000
Epoch:     3, Train Error: 0.74636, Validation Error: 1.05754 - Test Accuracy: 0.55385
Epoch:     4, Train Error: 0.30162, Validation Error: 1.17453 - Test Accuracy: 0.23077
Epoch:     5, Train Error: 0.16953, Validation Error: 1.23481 - Test Accuracy: 0.26923
Epoch:     6, Train 